In [ ]:
!pip install PyBioMed

In [ ]:
!pip install Pybel
!pip install RDkit

In [3]:
# Модуль для использования класса PyProtein в Colab
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive')

import Protein_funcs

Mounted at /content/drive


In [ ]:
import pandas as pd

#Аминокислотные последовательности получены из базы данных UniProt
cb2 = 'MEECWVTEIANGSKDGLDSNPMKDYMILSGPQKTAVAVLCTLLGLLSALENVAVLYLILSSHQLRRKPSYLFIGSLAGADFLASVVFACSFVNFHVFHGVDSKAVFLLKIGSVTMTFTASVGSLLLTAIDRYLCLRYPPSYKALLTRGRALVTLGIMWVLSALVSYLPLMGWTCCPRPCSELFPLIPNDYLLSWLLFIAFLFSGIIYTYGHVLWKAHQHVASLSGHQDRQVPGMARMRLDVRLAKTLGLVLAVLLICWFPVLALMAHSLATTLSDQVKKAFAFCSMLCLINSMVNPVIYALRSGEIRSSAHHCLAHWKKCVRGLGSEAKEEAPRSSVTETEADGKITPWPDSRDLDLSDC'
ckr5 = 'MDYQVSSPIYDINYYTSEPCQKINVKQIAARLLPPLYSLVFIFGFVGNMLVILILINCKRLKSMTDIYLLNLAISDLFFLLTVPFWAHYAAAQWDFGNTMCQLLTGLYFIGFFSGIFFIILLTIDRYLAVVHAVFALKARTVTFGVVTSVITWVVAVFASLPGIIFTRSQKEGLHYTCSSHFPYSQYQFWKNFQTLKIVILGLVLPLLVMVICYSGILKTLLRCRNEKKRHRAVRLIFTIMIVYFLFWAPYNIVLLLNTFQEFFGLNNCSSSNRLDQAMQVTETLGMTHCCINPIIYAFVGEKFRNYLLVFFQKHIAKRFCKCCSIFQQEAPERASSVYTRSTGEQEISVGL'
drd1 = 'MRTLNTSAMDGTGLVVERDFSVRILTACFLSLLILSTLLGNTLVCAAVIRFRHLRSKVTNFFVISLAVSDLLVAVLVMPWKAVAEIAGFWPFGSFCNIWVAFDIMCSTASILNLCVISVDRYWAISSPFRYERKMTPKAAFILISVAWTLSVLISFIPVQLSWHKAKPTSPSDGNATSLAETIDNCDSSLSRTYAISSSVISFYIPVAIMIVTYTRIYRIAQKQIRRIAALERAAVHAKNCQTTTGNGKPVECSQPESSFKMSFKRETKVLKTLSVIMGVFVCCWLPFFILNCILPFCGSGETQPFCIDSNTFDVFVWFGWANSSLNPIIYAFNADFRKAFSTLLGCYRLCPATNNAIETVSINNNGAAMFSSHHEPRGSISKECNLVYLIPHAVGSSEDLKKEEAAGIARPLEKLSPALSVILDYDTDVSLEKIQPITQNGQHPT'
b2ar = 'MGQPGNGSAFLLAPNGSHAPDHDVTQERDEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSLACADLVMGLAVVPFGAAHILMKMWTFGNFWCEFWTSIDVLCVTASIETLCVIAVDRYFAITSPFKYQSLLTKNKARVIILMVWIVSGLTSFLPIQMHWYRATHQEAINCYANETCCDFFTNQAYAIASSIVSFYVPLVIMVFVYSRVFQEAKRQLQKIDKSEGRFHVQNLSQVEQDGRTGHGLRRSSKFCLKEHKALKTLGIIMGTFTLCWLPFFIVNIVHVIQDNLIRKEVYILLNWIGYVNSGFNPLIYCRSPDFRIAFQELLCLRRSSLKAYGNGYSSNGNTGEQSGYHVEQEKENKLLCEDLPGTEDFVGHQGTVPSDNIDSQGRNCSTNDSLL'

proteins = [cb2, ckr5, drd1, b2ar]

cb2_mol_des = pd.read_csv('cb2_mol_des.csv')
ckr5_mol_des = pd.read_csv('ckr5_mol_des.csv')
drd1_mol_des = pd.read_csv('drd1_mol_des.csv')
b2ar_mol_des = pd.read_csv('b2ar_mol_des.csv')


molecular_descriptors = [cb2_mol_des, ckr5_mol_des, drd1_mol_des, b2ar_mol_des]

names = ['cb2_all_descriptors.csv', 'ckr5_all_descriptors.csv', 'drd1_all_descriptors.csv', 'b2ar_all_descriptors.csv']

counter = 0

new_datasets = []

for protein in proteins:
  cds = Protein_funcs.PyProtein(protein)
  result = cds.GetALL()
  df = pd.DataFrame(list(result.items()), columns=['Key', 'Value'])
  df = df[df['Key'].str.len() > 3]
  df.reset_index(drop=True, inplace=True)
  df['Key'] = df['Key'].str.replace('\d+', '')
  df['Key_prefix'] = df['Key'].str[:-1]
  aggregated_df = df.groupby('Key_prefix').agg({'Key': 'first', 'Value': 'mean'}).reset_index(drop=True)

  mol_des = molecular_descriptors[counter]

  n = 0
  for i in aggregated_df['Key']:
    mol_des[i] = aggregated_df['Value'][n]
    n += 1

  mol_des.to_csv(names[counter], index=False)

  new_datasets.append(mol_des)

  counter += 1


In [ ]:
for data in new_datasets:
  data.info()
  print(data)